In [5]:
PATH2DATASET = '/mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/'
DF_PATH = '/mnt/SSD/workspace/atomic_hack/df.csv'

In [6]:
import pandas as pd
import glob
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import utilities
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import timm
import math
from transformers import (get_linear_schedule_with_warmup, 
                          get_cosine_schedule_with_warmup, 
                          get_cosine_with_hard_restarts_schedule_with_warmup,
                          get_constant_schedule_with_warmup)
from tqdm import tqdm
import faiss
import random
import gc
import os
import transformers
from transformers import CLIPProcessor, CLIPVisionModel,  CLIPVisionConfig
from PIL import Image
from torchvision import transforms
from pytorch_metric_learning import losses
import open_clip
import sys

In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, window_size=None):
        self.length = 0
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.window_size = window_size

    def reset(self):
        self.length = 0
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        if self.window_size and (self.count >= self.window_size):
            self.reset()
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [8]:
class CFG:
    model_name = 'ViT-H-14-378-quickgelu' 
    model_data = 'dfn5b'
    samples_per_class = 50
    min_samples = 4
    image_size = 224 
    seed = 5
    workers = 6
    train_batch_size = 4
    valid_batch_size = 4
    emb_size = 512
    vit_bb_lr = {'10': 1.25e-6, '20': 2.5e-6, '26': 5e-6, '32': 10e-6} 
    vit_bb_wd = 1e-3
    hd_lr = 3e-4
    hd_wd = 1e-5
    autocast = True
    n_warmup_steps = 1000
    n_epochs = 50
    device = torch.device('cuda')
    s=30.
    m=.45
    m_min=.05
    acc_steps = 4
    global_step = 0
    n_classes = 5


In [9]:
utilities.set_seed(CFG.seed)

In [10]:
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_

In [11]:
df = pd.read_csv(DF_PATH)
df['fname'] = df['fname'].apply(lambda x: PATH2DATASET+x)
df.head()

,fname,x_center,y_center,width,height,x1,y1,x2,y2,label_idx,label,group,frame,split
0,/mnt/SSD/workspace/atomic_hack/Сварные_швы/dat...,0.524740,0.085781,0.202810,0.039953,914,252,1352,406,2,дефекты геометрии,8,63,train
1,/mnt/SSD/workspace/atomic_hack/Сварные_швы/dat...,0.512523,0.678613,0.249236,0.113984,837,2387,1376,2824,2,дефекты геометрии,8,63,train
2,/mnt/SSD/workspace/atomic_hack/Сварные_швы/dat...,0.464569,0.524089,0.159438,0.101058,831,1818,1175,2206,2,дефекты геометрии,11,79,train
3,/mnt/SSD/workspace/atomic_hack/Сварные_швы/dat...,0.587660,0.222092,0.163714,0.077556,1092,703,1446,1001,2,дефекты геометрии,6,94,train
4,/mnt/SSD/workspace/atomic_hack/Сварные_швы/dat...,0.463653,0.157462,0.108735,0.077556,884,455,1118,753,2,дефекты геометрии,6,94,train


In [12]:
# df['split'] = 1

In [13]:
# df.loc[df[df['group']==5].index, 'split'] = 0

In [14]:
# for path in np.random.choice(df.fname.unique(), size=250, replace=False):
    # df.loc[df[df['fname']==path].index, 'split'] = 0

In [15]:
from torch.utils.data import Dataset
from PIL import Image
import torch
import numpy as np
from tqdm import tqdm


class MetalDataset(Dataset):
    def __init__(self, 
                 df, 
                 mode='train', 
                 split = 0,
                 transform=None
                 ):
        # if mode=='train':
        #     self.df = df[df['split']!=split].reset_index(drop=True)
        # else:
        #     self.df = df[df['split']==split].reset_index(drop=True)
        self.df = df[df['split']==mode].reset_index(drop=True)
            
        self.augs = transform
        self.mode = mode
        self.cache_images()
        self.paths = list(self.df.fname.unique())

    def cache_images(self):
        self.idx2meta = {}
        idx = 0
        scale = 1.1
        for path in tqdm(self.df.fname.unique()):
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            bboxes  = df[df['fname']==path][['x1', 'y1', 'x2', 'y2']].values.tolist()
            labels = df[df['fname']==path]['label_idx'].values.tolist()

            for bbox, label in zip(bboxes, labels):
                x1, y1, x2, y2 = bbox
                x1 = int(x1/scale)
                x2 = int(x2*scale)    
                y1 = int(y1/scale)    
                y2 = int(y2*scale)  

                crop = img[y1:y2, x1:x2]
                
                self.idx2meta[idx] = {
                    'img': crop,
                    'label': label,
                }
                idx+=1

    def __len__(self):
        return len(self.idx2meta)
    
    def __getitem__(self, idx):
        item = self.idx2meta[idx]
        img = item['img']
        label = item['label']
                    
        try:
            if self.augs:
                img = self.augs(image=img)['image']
                img = torch.from_numpy(img).permute(2, 0, 1)
        except Exception as e:
            print(e, path)
            img = torch.zeros((3, 224, 224))
        
        return img, label

In [16]:
class Head(nn.Module):
    def __init__(self, hidden_size):
        super(Head, self).__init__()

        self.emb = nn.Linear(hidden_size, CFG.emb_size, bias=False)
        self.arc = utilities.ArcMarginProduct_subcenter(CFG.emb_size, CFG.n_classes)
        self.dropout = utilities.Multisample_Dropout()

    def forward(self, x):
        embeddings = self.dropout(x, self.emb)
        
        output = self.arc(embeddings)

        return output, F.normalize(embeddings)

In [17]:
class Model(nn.Module):
    def __init__(self, vit_backbone):
        super(Model, self).__init__()

        self.vit_backbone = vit_backbone
        with torch.no_grad():
            emb_size = vit_backbone(torch.randn(2, 3, image_size, image_size))[0].shape[1]
        self.head = Head(emb_size)

    def forward(self, images):

        # x = transforms.functional.resize(images, size=[CFG.image_size, CFG.image_size]) 
        # x = x/255
        # x = transforms.functional.normalize(x,  
        #                                      mean=model_transforms.transforms[-1].mean, 
        #                                      std=model_transforms.transforms[-1].std)

        x = self.vit_backbone.encode_image(images)
        
        return self.head(x)

    def get_parameters(self):

        parameter_settings = [] 
        parameter_settings.extend(self.get_parameter_section([(n, p) for n, p in self.vit_backbone.named_parameters()], lr=CFG.vit_bb_lr, wd=CFG.vit_bb_wd)) 

        parameter_settings.extend(self.get_parameter_section([(n, p) for n, p in self.head.named_parameters()], lr=CFG.hd_lr, wd=CFG.hd_wd)) 

        return parameter_settings

    def get_parameter_section(self, parameters, lr=None, wd=None): 
        parameter_settings = []


        lr_is_dict = isinstance(lr, dict)
        wd_is_dict = isinstance(wd, dict)

        layer_no = None
        for no, (n,p) in enumerate(parameters):
            
            for split in n.split('.'):
                if split.isnumeric():
                    layer_no = int(split)
            
            if not layer_no:
                layer_no = 0
            
            if lr_is_dict:
                for k,v in lr.items():
                    if layer_no < int(k):
                        temp_lr = v
                        break
            else:
                temp_lr = lr

            if wd_is_dict:
                for k,v in wd.items():
                    if layer_no < int(k):
                        temp_wd = v
                        break
            else:
                temp_wd = wd

            weight_decay = 0.0 if 'bias' in n else temp_wd

            parameter_setting = {"params" : p, "lr" : temp_lr, "weight_decay" : temp_wd}

            parameter_settings.append(parameter_setting)

            #print(f'no {no} | params {n} | lr {temp_lr} | weight_decay {weight_decay} | requires_grad {p.requires_grad}')

        return parameter_settings


In [18]:
def get_augs(mean, std, image_size, mode='train'):
    if mode=='train':
        return A.Compose([
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.ImageCompression(quality_lower=10, quality_upper=100, p=0.7),
                # A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
                A.Resize(image_size, image_size),
                # A.Cutout(max_h_size=int(image_size * 0.4), max_w_size=int(image_size * 0.4), num_holes=1, p=0.5),
                # A.OneOf([
                #     A.ChannelShuffle(),
                #     A.ChannelDropout(),
                #     A.ColorJitter(),
                #     A.ToGray(),
                # ], p=0.65),
                A.Normalize(mean=mean, std=std, p=1), 
            ])

    else:
        return A.Compose([
            A.Resize(image_size, image_size),
            A.Normalize(mean=mean, std=std, p=1)
        ])

In [19]:
vit_backbone, model_transforms, _ = open_clip.create_model_and_transforms(CFG.model_name, pretrained=CFG.model_data)

mean, std = model_transforms.transforms[-1].mean, model_transforms.transforms[-1].std
image_size = model_transforms.transforms[0].size[0]

model = Model(vit_backbone.cpu()).to(CFG.device)

In [20]:
train_dataset = MetalDataset(df, mode='train', split=0, transform=get_augs(mean, std, image_size, 'train'))
valid_dataset = MetalDataset(df, mode='val', split=0, transform=get_augs(mean, std, image_size, 'val'))

100%|█████████████████████████████████████████| 115/115 [00:04<00:00, 26.50it/s]


In [21]:
train_loader = DataLoader(train_dataset, batch_size = CFG.train_batch_size, num_workers=CFG.workers, shuffle=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_batch_size, num_workers=CFG.workers, shuffle=False, drop_last=False)

In [22]:
def ArcFace_criterion(logits_m, target, margins):
    arc = utilities.ArcFaceLossAdaptiveMargin(margins=margins, s=CFG.s)
    loss_m = arc(logits_m, target, CFG.n_classes)
    return loss_m

In [23]:
def train(model, train_loader, optimizer, scaler, scheduler, epoch, pbar_draw=False):
    model.train()
    loss_metrics = AverageMeter()
    criterion = ArcFace_criterion

    tmp = np.sqrt(1 / np.sqrt(value_counts))
    margins = (tmp - tmp.min()) / (tmp.max() - tmp.min()) * CFG.m + CFG.m_min
        
    bar = tqdm(train_loader, disable=not pbar_draw)
#     bar = tqdm(train_loader, disable=False)
    for step, data in enumerate(bar):
        step += 1
        images = data[0].to(CFG.device, dtype=torch.float)
        labels = data[1].to(CFG.device)
        batch_size = labels.size(0)

        with torch.cuda.amp.autocast(enabled=CFG.autocast):
            outputs, features = model(images)

        loss = criterion(outputs, labels, margins)
        loss_metrics.update(loss.item(), batch_size)
        loss = loss / CFG.acc_steps
        scaler.scale(loss).backward()

        if step % CFG.acc_steps == 0 or step == len(bar):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
            CFG.global_step += 1
            
        lrs = get_lr_groups(optimizer.param_groups)

        loss_avg = loss_metrics.avg

        bar.set_postfix(loss=loss_avg, epoch=epoch, lrs=lrs, step=CFG.global_step)

In [24]:
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
def validate(model, loader):
    model.eval()
    device = 'cuda'
    preds = []
    labels = []
    for i, (images, label) in enumerate(loader):
        with torch.no_grad():
            outputs = model(images.to(device))[0].argmax(1).cpu().detach().tolist()
            preds.extend(outputs)
            labels.extend(label)
            
    metric = f1_score(labels, preds, average='macro')
    return metric

In [25]:
value_counts = df['label_idx'].value_counts().tolist()

In [26]:
#model.get_parameters()

In [27]:
def get_lr_groups(param_groups):
        groups = sorted(set([param_g['lr'] for param_g in param_groups]))
        groups = ["{:2e}".format(group) for group in groups]
        return groups

In [ ]:
optimizer = torch.optim.AdamW(model.get_parameters())
scaler = torch.cuda.amp.GradScaler(enabled=CFG.autocast)
steps_per_epoch = math.ceil(len(train_loader) / CFG.acc_steps)
num_training_steps = math.ceil(CFG.n_epochs * steps_per_epoch)
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_training_steps=num_training_steps,
                                            num_warmup_steps=CFG.n_warmup_steps)   
CFG.global_step = 0    
best_score = 0
for epoch in range(math.ceil(CFG.n_epochs)):

    train(model, train_loader, optimizer, scaler, scheduler, epoch, pbar_draw=True)
    score = validate(model, valid_loader)
    print(f'Epoch = {epoch}, score:', score)
    # if score>0.97:
    #     torch.save(model.state_dict(), f'{CFG.model_name}_{CFG.model_data}_{fold_idx}_{score}.pth')
    gc.collect()
    torch.cuda.empty_cache()
    if score>best_score and score>0.72:
        best_score = score
        # torch.save(model.state_dict(), f'best_model_{round(best_score, 3)}.pt')
        torch.save(model.state_dict(), f'/mnt/SSD/workspace/atomic_hack/{CFG.model_name}_{CFG.model_data}_{score}.pth')

  0%|          | 0/731 [00:00<?, ?it/s]

/home/ivan/anaconda3/envs/env3.12/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [ ]:
# model_name = CFG.model_name.replace('/','-')
# torch.save(model.state_dict(), f'../models/{model_name}')

In [ ]:
model.eval()
device = 'cuda'
preds = []
labels = []
for i, (images, label) in enumerate(valid_loader):
    with torch.no_grad():
        outputs = model(images.to(device))[0].argmax(1).cpu().detach().tolist()
        preds.extend(outputs)
        labels.extend(label)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(labels, preds)

In [ ]:
idx2label = {
    0: 'прилегающие дефекты', 
    1: 'дефекты целостности',
    2: 'дефекты геометрии',
    3: 'дефекты постобработки',
    4: 'дефекты невыполнения'
}

labels = list(idx2label.values())
labels

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(cm, labels, labels)
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='.3g') # font siz